# Naive Bayes Spam Filter
### Carlos Marcelo Barrera Nolasco
### 101893

* Procedure
    * Divide data in train and test sets
    * Keep test data in a safe!
    * Transform test data (normalize, discretize, etc)
    * Train model
    * Transform test data with the parameters found in step 3
    * Test model with test data
    * Evaluate results

In [2]:
import pandas as pd
import numpy as np
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from random import random

Perhaps easiest way to read in data is using Pandas. Pandas is a library for manipulating data. Similar to R's dataframes and very useful albeit in some cases confusing to combine with other libraries:

In [3]:
df = pd.read_csv("../../Recursos/Spam/spambase/spambase.data",header=None)

In [4]:
# This data does not have headers so each attribute or field is simply enumerated
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


There are a few ways to split data into train and test. The first is using Sklearn, which is a machine learning library in python has a method for spliting data into train and test

In [6]:
# Here df.columns is a list of all the columns and df.columns[0:-1] is all columns minus the last which is y. 
# If the data had headers you could use column names: df[['column1','column2','etc']]
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[df.columns[-1]], train_size=0.75)

Something important to note. Sklearn is able to take in pandas dataframes but returns arrays.

The other way to split data that is useful to know is:

In [7]:
# index for selecting data 0.75 is the percentage in training
index=np.array([1 if random() < 0.75 else 0 for i in range(len(df))])

In [8]:
# Separate both train and test as well as the response variable
X_train=np.array(df[df.columns[0:-1]])[index==1]
X_test=np.array(df[df.columns[0:-1]])[index==0]
Y_train=np.array(df[df.columns[-1]])[index==1]
Y_test=np.array(df[df.columns[-1]])[index==0]

The above method for spliting data can also be used for selecting a subset of an array using the values of an equally sized array. Useful for the current excercise. For example, to extract all instances of spam for the training data: 

In [9]:
# Normalizar no ayuda mucho pero sale igual al de sklearn. Para que las alturas del pdf signifiquen lo mismo 
scaler = preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

## Calculating the probabilities

In [10]:
totSpam = 0
for i in range(0, len(Y_train)):
    if Y_train[i]==1:
        totSpam +=1
probSpam = totSpam/len(Y_train)
probNoSpam = 1-probSpam

## Calculating means and standard deviations

In [12]:
mediaSpam = np.mean(X_train[Y_train==1], axis=0)
dsSpam = np.std(X_train[Y_train==1], axis=0)
mediaNoSpam = np.mean(X_train[Y_train==0], axis=0)
dsNoSpam = np.std(X_train[Y_train==0], axis=0)

## Classifier

In [13]:
def classifier(row):
    min = len(row)*-10
    isSpam = np.log(probSpam)
    notSpam = np.log(probNoSpam)
    for i in range (0, len(row)):
        isSpam += np.log(norm(mediaSpam[i],dsSpam[i]).pdf(row[i])) if norm(mediaSpam[i],dsSpam[i]).pdf(row[i]) > 0 else min
        notSpam += np.log(norm(mediaNoSpam[i],dsNoSpam[i]).pdf(row[i])) if norm(mediaNoSpam[i],dsNoSpam[i]).pdf(row[i]) > 0 else min
    return 1 if isSpam > notSpam else 0 

## Prediction over the test set

In [15]:
predicts=[]
for row in (X_test):
    predicts.append(classifier(row))

## Confusion Matrix

In [16]:
confusion_matrix(Y_test, predicts)

array([[476, 206],
       [ 14, 424]])

## Using Scikit Learn

In [17]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
predict = gnb.predict(X_test)
confusion_matrix(Y_test, predict)

array([[476, 206],
       [ 14, 424]])